# Cold Face Test – Plots for Graphical Abstract

In [ ]:
import json
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

from fau_colors import cmaps, colors_all
import biopsykit as bp
from biopsykit.protocols import MIST
from biopsykit.utils.dataframe_handling import multi_xs

from cft_analysis.datasets import CftDatasetProcessed

from IPython.display import Markdown

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="talk", style="ticks", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

## Data Import

In [ ]:
# get path to analysis results
base_path = Path("../../data")

In [ ]:
results_path = base_path.joinpath("../results")
plot_path = results_path.joinpath("plots_graphical_abstract")
bp.utils.file_handling.mkdirs([results_path, plot_path])

In [ ]:
hue_order = ["Control", "CFT"]

subphase_order = ["BL", "RP/CFI", "AT", "FB"]

In [ ]:
dataset = CftDatasetProcessed(base_path, exclude_subjects=True)
dataset

In [ ]:
mist = MIST.from_file(base_path.joinpath("mist_cft.json"))
mist

In [ ]:
mist.add_saliva_data(dataset.cortisol, "cortisol", sample_times=dataset.sample_times)

In [ ]:
hr_ensemble = dataset.heart_rate_ensemble
hr_mean = dataset.heart_rate
hr_mean = hr_mean.xs("HR_Norm", level="type")
hr_mean.head()

In [ ]:
mist.add_hr_ensemble("hr_ensemble", {key: val.xs(key) for key, val in hr_ensemble.groupby("phase")})

hr_mean = hr_mean.rename({"RP_CFI": "RP/CFI"}, level="subphase").reindex(subphase_order, level="subphase")
mist.add_hr_results("hr_mean", hr_mean)

## HR Ensemble Plot

In [ ]:
fig, ax = plt.subplots()

# HR Ensemble Plot
mist.hr_ensemble_plot("hr_ensemble", ax=ax)

#for path in [plot_path, paper_img_path]:
#    if path is not None:
#        fig.savefig(path.joinpath("img_hr_ensemble.pdf"), transparent=True)

## HR Mean Plot

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

# HR Mean Plot
mist.hr_mean_plot("hr_mean", is_relative=True, ax=ax, order=hue_order)

fig.savefig(plot_path.joinpath("img_hr_mean_course.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

mist.saliva_plot(
    "cortisol",
    legend_loc="upper right",
    legend_fontsize="medium",
    hue_order=hue_order,
    linestyle=["-", "--"],
    marker=["o", "P"],
    ax=ax,
)

fig.savefig(plot_path.joinpath("img_cortisol_response.pdf"), transparent=True)